In [19]:
from transformers import AutoModelForCausalLM, AutoConfig


# create an instance of AutoModelForCausalLM from a configuration file without loading the weights
config = AutoConfig.from_pretrained('/models/mistral-100M/')
config

MistralConfig {
  "_name_or_path": "/models/mistral-100M/",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 16,
  "hidden_act": "silu",
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 768,
  "max_position_embeddings": 1024000,
  "model_type": "mistral",
  "num_attention_heads": 8,
  "num_hidden_layers": 4,
  "num_key_value_heads": 2,
  "pad_token_id": 10,
  "rms_norm_eps": 1e-05,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.46.3",
  "unsloth_version": "2024.8",
  "use_cache": true,
  "vocab_size": 131072
}

In [20]:
model = AutoModelForCausalLM.from_config(config)
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(131072, 256, padding_idx=10)
    (layers): ModuleList(
      (0-3): 4 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=256, out_features=128, bias=False)
          (k_proj): Linear(in_features=256, out_features=32, bias=False)
          (v_proj): Linear(in_features=256, out_features=32, bias=False)
          (o_proj): Linear(in_features=128, out_features=256, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=256, out_features=768, bias=False)
          (up_proj): Linear(in_features=256, out_features=768, bias=False)
          (down_proj): Linear(in_features=768, out_features=256, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((256,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((256,), eps=1e-05)
      )
    

In [21]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

count_parameters(model)

69798144

## initialize parameters and save

In [23]:
import torch


# initialize parameters for the model
for p in model.parameters():
    if p.dim() > 1:
        torch.nn.init.xavier_uniform_(p, gain=(config.num_hidden_layers * 2) ** -0.5)


In [25]:
model.bfloat16()

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(131072, 256, padding_idx=10)
    (layers): ModuleList(
      (0-3): 4 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=256, out_features=128, bias=False)
          (k_proj): Linear(in_features=256, out_features=32, bias=False)
          (v_proj): Linear(in_features=256, out_features=32, bias=False)
          (o_proj): Linear(in_features=128, out_features=256, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=256, out_features=768, bias=False)
          (up_proj): Linear(in_features=256, out_features=768, bias=False)
          (down_proj): Linear(in_features=768, out_features=256, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((256,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((256,), eps=1e-05)
      )
    

In [26]:
model.save_pretrained('/models/mistral-100M')

---

In [34]:
from transformers import AutoModelForCausalLM, AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("/models/train/20250122-mistral-100m-pretrain")
model = AutoModelForCausalLM.from_pretrained("/models/train/20250122-mistral-100m-pretrain")

tokenizer, model

(PreTrainedTokenizerFast(name_or_path='/models/train/20250122-mistral-100m-pretrain', vocab_size=131072, model_max_length=1024000, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
 	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	3: AddedToken("[INST]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	4: AddedToken("[/INST]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	5: AddedToken("[AVAILABLE_TOOLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	6:

In [35]:
input_ids = tokenizer.encode("Hello, ", return_tensors="pt")
input_ids

tensor([[    1, 22177,  1044,  1032]])

In [36]:
output_ids = model.generate(input_ids)
output_ids

tensor([[    1, 22177,  1044,  1032,  1050,  1056,  1053,  1052,  1053,  1056,
          1051,  1053,  1052,  1049,  1052,  1462,  2143,  3039,  1044,  1032]])

In [37]:
tokenizer.decode(output_ids[0])

'<s>Hello, 28545835414 - your way, '